In [1]:
from tensorflow import keras

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 1s 0us/step


In [5]:
X_val = X_train_full[:5000]
y_val = y_train_full[:5000]
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]

In [66]:
X_train.shape

(55000, 28, 28)

In [68]:
from tensorflow.train import BytesList, Int64List
from tensorflow.train import Feature, Features, Example
import numpy as np
import os

def make_proto_example(example, label):
    return Example(
        features=Features(
            feature={
                "example": Feature(bytes_list=BytesList(value=[tf.io.serialize_tensor(example).numpy()])),
                "label": Feature(int64_list=Int64List(value=[label]))
            }))

def write_to_files(examples, labels, num_files, location):
    if not os.path.exists(location):
        os.makedirs(location)
    file_num = 0
    
    for batch in np.split(np.array(list(zip(examples, labels))), num_files):
        with tf.io.TFRecordWriter(location + f"dataset_{file_num}.tfrecord") as f:
            for item in batch:
                example = make_proto_example(item[0], item[1])
                f.write(example.SerializeToString())
        file_num += 1

In [69]:
num_files = 100
location = "./datasets/fashion_minst/train/"

write_to_files(X_train, y_train, num_files, location)

In [70]:
write_to_files(X_val, y_val, 1, "./datasets/fashion_minst/val/")

In [71]:
write_to_files(X_test, y_test, 1, "./datasets/fashion_minst/test/")

In [76]:
def read_example(tfrecord):
    feature_description = {
        "example": tf.io.FixedLenFeature([], tf.string, default_value=""),
        "label": tf.io.FixedLenFeature([], tf.int64, default_value=-1)
    }
    example = tf.io.parse_single_example(tfrecord, feature_description)
    image = tf.io.parse_tensor(example["example"], out_type=tf.uint8)
    image = tf.reshape(image, shape=[28, 28])
    return (image, example["label"])

def read_tfrecord_files(location, num_files):
    files = [location + f"dataset_{file_num}.tfrecord" for file_num in range(0, num_files)]
    dataset = tf.data.TFRecordDataset(files)
    dataset = dataset.shuffle(1000)
    dataset = dataset.map(read_example)
    dataset = dataset.batch(32).prefetch(1)
    
    return dataset

In [80]:
num_files = 100
location = "./datasets/fashion_minst/train/"

new_train_data = read_tfrecord_files(location, num_files)
new_val_data = read_tfrecord_files("./datasets/fashion_minst/val/", 1)
new_test_data = read_tfrecord_files("./datasets/fashion_minst/test/", 1)

In [89]:
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())

data_sample = np.concatenate(list(new_train_data.take(100).map(lambda example, label: example).as_numpy_iterator()))

standardization = Standardization(input_shape=[28,28])

standardization.adapt(data_sample)

model = keras.models.Sequential()
model.add(standardization)
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="nadam", metrics=["accuracy"])

In [95]:
from datetime import datetime
logs = os.path.join(os.curdir, "training_logs", "run_" + datetime.now().strftime("%Y%m%d_%H%M%S"))

tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=logs, histogram_freq=1, profile_batch=10)

model.fit(new_train_data, epochs=50, validation_data=new_val_data, callbacks=[tensorboard_cb])

Epoch 1/50
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2158 - accuracy: 0.9188 - val_loss: 0.3595 - val_accuracy: 0.8850
Epoch 2/50
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1984 - accuracy: 0.9255 - val_loss: 0.3897 - val_accuracy: 0.8866
Epoch 3/50
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1826 - accuracy: 0.9309 - val_loss: 0.4206 - val_accuracy: 0.8848
Epoch 4/50
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1731 - accuracy: 0.9340 - val_loss: 0.3864 - val_accuracy: 0.8896
Epoch 5/50
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1574 - accuracy: 0.9403 - val_loss: 0.3674 - val_accuracy: 0.8898
Epoch 6/50
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1523 - accuracy: 0.9430 - val_loss: 0.4167 - val_accuracy: 0.8892
Epoch 7/50
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1372 - accuracy: 0.9488 - val_loss: 0.4377 - val_accuracy:

In [91]:
%load_ext tensorboard

In [94]:
%tensorboard --logdir=./training_logs --port=6008

Launching TensorBoard...

KeyboardInterrupt: 